In [1]:
import sys
from pysc2.lib import app
from argparse import Namespace

# Parse the flags so that the default values are ready.
app.FLAGS([sys.argv[0]])

from pysc2 import maps
from pysc2.env import available_actions_printer
from pysc2.env import run_loop
from pysc2.env import sc2_env
from pysc2.agents.base_agent import *
from pysc2.agents.random_agent import *
from pysc2.agents.scripted_agent import *
from pysc2.lib import actions

import threading
from Queue import Queue

In [2]:
CMD_GET = 0    # get observations.
CMD_PUT = 1    # put actions.

In [3]:
# Inherit from the existing agents so that 
# we can call their step functions for testing purpose.
class NotebookAgent(CollectMineralShards, MoveToBeacon, DefeatRoaches):
  def __init__(self):
    self.qi, self.qo = Queue(), Queue()
    self.noop = actions.FunctionCall(0, [])

  def step(self, obs):
    BaseAgent.step.im_func(self, obs)
    if not self.qi.empty():
      c = self.qi.get()
      if c.cmd == CMD_GET:
        self.qo.put(obs)
        if c.pause:
          c_resume = self.qi.get()
          self.qo.put(None)
          return c_resume.act
      elif c.cmd == CMD_PUT:
        self.qo.put(None)
        return c.act
    return self.noop

  # Put an action for execution.
  def put(self, act):
    self.qi.put(Namespace(cmd=CMD_PUT, act=act))
    self.qo.get()

  # Get the current observation.
  # If pause, use put to resume.
  def get(self, pause=False):
    self.qi.put(Namespace(cmd=CMD_GET, pause=pause))
    return self.qo.get()

  # Launches the sc2 engine.
  def run_thread(self):
    with sc2_env.SC2Env(
      map_name="CollectMineralShards", step_mul=1, visualize=True) as env:
      env = available_actions_printer.AvailableActionsPrinter(env)
      self.env = env
      run_loop.run_loop([self], env, max_frames=2500 * 8)

In [4]:
agent = NotebookAgent()

In [5]:
t = threading.Thread(target=agent.run_thread)
t.start()

   0/no_op                                              ()
   1/move_camera                                        (1/minimap [64, 64])
   2/select_point                                       (6/select_point_act [4]; 0/screen [64, 64])
   3/select_rect                                        (7/select_add [2]; 0/screen [64, 64]; 2/screen2 [64, 64])
   4/select_control_group                               (4/control_group_act [5]; 5/control_group_id [10])
   7/select_army                                        (7/select_add [2])


In [7]:
# Runs the collect mineral action once, manually.
# You can inspect the observations before issuing the actions.
obs = agent.get()
act = CollectMineralShards.step.im_func(agent, obs)
agent.put(act)

In [10]:
# This gets the observation and blocks the execution
obs = agent.get(pause=True)

In [11]:
# This resumes the game.
act = CollectMineralShards.step.im_func(agent, obs)
agent.put(act)

Took 30.674 seconds for 1114 steps: 36.317 fps
